https://www.kaggle.com/c/state-farm-distracted-driver-detection/overview

In [1]:
# Hello this is the RAM optimised branch

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
# import cv2
# from sklearn.model_selection import train_test_split
from PIL import Image
# import torchvision
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm.notebook import trange, tqdm

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Data Preprocessing

In [4]:
class DriverDataset(Dataset):
    def __init__(self, dir_path, train): # The last parameter is used to split up the data into train or inference
        self.train = train
        
        self.x_ = []  
        self.y_ = []  
        if self.train:
            for i in range(10):
                sub_dir_path =f'{dir_path}/c{i}'
                files = os.listdir(sub_dir_path)
                for img_name in files:
                    full_file_path = sub_dir_path + '/' + img_name
                    self.x_.append(full_file_path)
                    self.y_.append(i)
        else:
            files = os.listdir(dir_path)
            for img_name in files:
                    full_file_path = dir_path + '/' + img_name
                    self.x_.append(full_file_path)
                    # self.y_.append(i)
    
    def __len__(self):
        return len(self.x_)
    
    def __getitem__(self, index):
        x_image = np.array(Image.open(self.x_[index]))
        # print(x_image.shape)
        x_image = x_image.reshape( (3, 480, 640) )
        if self.train: return x_image, self.y_[index]
        return x_image

In [5]:
dataset = DriverDataset('data/imgs/train', True)
train_share = 0.8
train_dataset, val_dataset = random_split(dataset, [int(train_share*len(dataset)),len(dataset) -int(train_share*len(dataset)) ])
test_dataset = DriverDataset('data/imgs/test', False)



train_loader = DataLoader(train_dataset,batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=512, shuffle=False)

## Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.l1 = nn.Linear(16*158*118,640)
        self.l2 = nn.Linear(640, 160)
        self.l3 = nn.Linear(160, 10)
    
    def forward(self, x):
        # x -> (n, 3, 640, 480)
        out = self.pool(F.relu(self.conv1(x))) # -> (n, 6, 318, 238)
        out = self.pool(F.relu(self.conv2(out))) # -> (n, 16, 158, 118)
        out = out.view(-1, 16*158*118)
        out = F.relu(self.l1(out))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out

model = CNN().to(device)

In [7]:
loss_category = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [8]:
model = torch.compile(model)

In [9]:
torch.set_float32_matmul_precision('high')

In [10]:
for epoch in trange(1):
    for i, (x_, y_) in enumerate(train_loader):
        x_ = x_.to(torch.float32).to(device)
        y_ = F.one_hot(y_.clone().detach().long(), 10).float().to(device)
        y_pred = model(x_)
        loss = loss_category(y_pred, y_)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()

    if (epoch+1)%5 == 0: print(f'Epoch {epoch+1}: {loss.item():3f}')

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
model.eval()
score = 0
tot = 0
with torch.no_grad():
    for i, (x_, y_) in enumerate(tqdm(train_loader, desc="Processing", leave=False)):
        x_ = x_.to(torch.float32).to(device)
        y_ = y_.to(device)
        y_pred = model(x_)
        for y1, y_pred1 in zip(y_.int(), torch.argmax(y_pred, dim=1)):
            if y1.item() == y_pred1.item():
                score += 1
            tot += 1
print('Train acc:', score * 100 / tot, '%')


Processing:   0%|          | 0/36 [00:00<?, ?it/s]

Train acc: 10.32387535537098 %


In [12]:
model.eval()
score = 0
tot = 0
with torch.no_grad():
    for i, (x_, y_) in enumerate(tqdm(val_loader, desc="Processing", leave=False)):
        x_ = x_.to(torch.float32).to(device)
        y_ = y_.to(device)
        y_pred = model(x_)
        for y1, y_pred1 in zip(y_.int(), torch.argmax(y_pred, dim=1)):
            if y1.item() == y_pred1.item():
                score += 1
            tot += 1
print('Train acc:', score * 100 / tot, '%')

Processing:   0%|          | 0/9 [00:00<?, ?it/s]

Train acc: 11.014492753623188 %


Best acc: 75.6 % 

## Inference

In [13]:

model.eval()
out = []
with torch.no_grad():
    for x_ in tqdm(test_loader, desc="Processing"):
        x_ = x_.to(torch.float32).to(device)
        y_pred = model(x_)
        softmax_output = F.softmax(y_pred, dim=1)
        out.extend(list(softmax_output))


Processing:   0%|          | 0/156 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
len(test_dataset.x_)

In [ ]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []

for i in out:
    c0.append(i[0].item())
    c1.append(i[1].item())
    c2.append(i[2].item())
    c3.append(i[3].item())
    c4.append(i[4].item())
    c5.append(i[5].item())
    c6.append(i[6].item())
    c7.append(i[7].item())
    c8.append(i[8].item())
    c9.append(i[9].item())

In [ ]:
out = {'img':[img.split('/')[-1] for img in test_dataset.x_], 
       'c0': c0,
       'c1': c1,
       'c2': c2,
       'c3': c3,
       'c4': c4,
       'c5': c5,
       'c6': c6,
       'c7': c7,
       'c8': c8,
       'c9': c9,
       }
df = pd.DataFrame(out)
df.to_csv('data/output.csv', index=False)